In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from src.utils.data_loader import load_data
import numpy as np
from sklearn.model_selection import train_test_split
import cv2
import time

# Carga el modelo existente
model_path = os.path.join(os.path.dirname(os.path.abspath('')), 'data', 'models', 'modelo_formas.h5')
model = tf.keras.models.load_model(model_path)
print(f"Modelo cargado desde: {model_path}")

# Carga las clases existentes para mantener consistencia
data_dir = os.path.join(os.path.dirname(os.path.abspath('')), 'data', 'raw')
_, _, forma_to_label = load_data(data_dir, max_images_per_class=0)  # Solo carga las clases
num_classes = len(forma_to_label)
print("Clases originales:", forma_to_label)

# Filtra las clases al conjunto de 16 especificadas
selected_classes = ['circle', 'cone', 'cube', 'cuboid', 'cylinder', 'ellipse', 'hexagon', 'octagon', 'pentagon', 'prism', 'pyramid', 'rectangle', 'rhombus', 'sphere', 'square', 'triangle']
forma_to_label = {forma: idx for idx, forma in enumerate(selected_classes) if forma in forma_to_label}
num_classes = len(forma_to_label)
print("Clases seleccionadas:", forma_to_label)

# Función para capturar y seleccionar un objeto con cámara
def capture_and_select_object():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: No se pudo abrir la cámara.")
        return None

    print("Cámara activada. Dibuja un rectángulo alrededor del objeto con el mouse y presiona 'c' para capturar. Presiona 'q' para salir.")
    roi = None
    rect = (0, 0, 0, 0)
    drawing = False

    def draw_rectangle(event, x, y, flags, param):
        nonlocal rect, drawing, roi
        if event == cv2.EVENT_LBUTTONDOWN:
            drawing = True
            rect = (x, y, 0, 0)
        elif event == cv2.EVENT_MOUSEMOVE and drawing:
            rect = (rect[0], rect[1], x - rect[0], y - rect[1])
        elif event == cv2.EVENT_LBUTTONUP:
            drawing = False
            rect = (rect[0], rect[1], x - rect[0], y - rect[1])
            roi = frame[rect[1]:rect[1]+rect[3], rect[0]:rect[0]+rect[2]]
            if roi.size > 0:
                roi = cv2.resize(roi, (128, 128))
                cv2.imshow('ROI Seleccionada', roi)

    cv2.namedWindow('Selecciona Objeto')
    cv2.setMouseCallback('Selecciona Objeto', draw_rectangle)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if drawing:
            cv2.rectangle(frame, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 2)
        cv2.imshow('Selecciona Objeto', frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('c') and roi is not None:
            break
        elif key == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            return None

    cap.release()
    cv2.destroyAllWindows()
    return roi

# Bucle para capturar, verificar y recolectar datos antes de reentrenar
new_images = []
new_labels = []
print("Comenzando reentrenamiento interactivo. Captura objetos con la cámara y verifica las predicciones. Presiona 'q' para salir y reentrenar con todas las imágenes capturadas.")
while True:
    roi = capture_and_select_object()
    if roi is None:
        break

    # Normaliza la ROI
    roi_normalized = roi / 255.0

    # Predice con el modelo
    prediction = model.predict(np.expand_dims(roi_normalized, axis=0), verbose=0)
    clase = np.argmax(prediction)
    probabilidad = np.max(prediction) * 100
    forma_predicha = next((forma for forma, idx in forma_to_label.items() if idx == clase), "Desconocida")
    print(f"Forma predicha: {forma_predicha} (Confianza: {probabilidad:.2f}%)")

    # Pide retroalimentación
    feedback = input("¿Es correcto? (s/n, o 'q' para salir y reentrenar): ").lower()
    if feedback == 'q':
        break
    elif feedback == 's':
        label = clase
        forma_correcta = forma_predicha
    else:
        print("Selecciona la forma correcta (0-15):", forma_to_label)
        try:
            label = int(input("Ingresa el número de la forma correcta: "))
            if label not in range(num_classes):
                print("Número inválido. Ignorando esta imagen.")
                continue
            forma_correcta = next((forma for forma, idx in forma_to_label.items() if idx == label), "Desconocida")
        except ValueError:
            print("Entrada inválida. Ignorando esta imagen.")
            continue

    # Guarda la imagen en la carpeta correspondiente
    save_dir = os.path.join(data_dir, forma_correcta)
    os.makedirs(save_dir, exist_ok=True)
    image_name = f"{forma_correcta}_{len(os.listdir(save_dir)) + 1}.jpg"
    image_path = os.path.join(save_dir, image_name)
    cv2.imwrite(image_path, roi)
    print(f"Imagen guardada en: {image_path}")

    # Añade a los datos nuevos
    new_images.append(roi_normalized)
    new_labels.append(label)

# Si hay nuevas imágenes, reentrena el modelo solo si hay suficientes datos
if new_images and len(new_images) > 1:  # Asegura al menos 2 imágenes
    new_images = np.array(new_images)
    new_labels = tf.keras.utils.to_categorical(new_labels, num_classes)  # Asegura 16 clases

    # Divide los nuevos datos
    X_train_new, X_val_new, y_train_new, y_val_new = train_test_split(new_images, new_labels, test_size=0.2, random_state=42)

    # Augmentación
    datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)
    train_generator = datagen.flow(X_train_new, y_train_new, batch_size=16)
    validation_generator = datagen.flow(X_val_new, y_val_new, batch_size=16)

    # Recompila si es necesario
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Reentrena el modelo
    print("Reentrenando modelo con nuevas imágenes...")
    model.fit(train_generator, epochs=5, validation_data=validation_generator)

    # Guarda el modelo actualizado
    model.save(model_path)
    print(f"Modelo reentrenado y guardado en: {model_path}")
else:
    print("No se capturaron suficientes imágenes para reentrenar. Se necesitan al menos 2 imágenes.")

Modelo cargado desde: D:\Descargas 2.0\SEMESTREVII\InteraccionHumanoComputador\PrimeraTarea\AnalisiOfSolidAndPlane\data\models\modelo_formas.h5
Clases originales: {'circle': 0, 'cone': 1, 'cube': 2, 'cuboid': 3, 'cylinder': 4, 'ellipse': 5, 'hexagon': 6, 'octagon': 7, 'pentagon': 8, 'prism': 9, 'pyramid': 10, 'rectangle': 11, 'rhombus': 12, 'sphere': 13, 'square': 14, 'triangle': 15}
Clases seleccionadas: {'circle': 0, 'cone': 1, 'cube': 2, 'cuboid': 3, 'cylinder': 4, 'ellipse': 5, 'hexagon': 6, 'octagon': 7, 'pentagon': 8, 'prism': 9, 'pyramid': 10, 'rectangle': 11, 'rhombus': 12, 'sphere': 13, 'square': 14, 'triangle': 15}
Comenzando reentrenamiento interactivo. Captura objetos con la cámara y verifica las predicciones. Presiona 'q' para salir y reentrenar con todas las imágenes capturadas.
Cámara activada. Dibuja un rectángulo alrededor del objeto con el mouse y presiona 'c' para capturar. Presiona 'q' para salir.
Forma predicha: circle (Confianza: 99.40%)


¿Es correcto? (s/n, o 'q' para salir y reentrenar):  s


Imagen guardada en: D:\Descargas 2.0\SEMESTREVII\InteraccionHumanoComputador\PrimeraTarea\AnalisiOfSolidAndPlane\data\raw\circle\circle_2618.jpg
Cámara activada. Dibuja un rectángulo alrededor del objeto con el mouse y presiona 'c' para capturar. Presiona 'q' para salir.
Forma predicha: rectangle (Confianza: 58.11%)


¿Es correcto? (s/n, o 'q' para salir y reentrenar):  s


Imagen guardada en: D:\Descargas 2.0\SEMESTREVII\InteraccionHumanoComputador\PrimeraTarea\AnalisiOfSolidAndPlane\data\raw\rectangle\rectangle_2561.jpg
Cámara activada. Dibuja un rectángulo alrededor del objeto con el mouse y presiona 'c' para capturar. Presiona 'q' para salir.
Forma predicha: cube (Confianza: 94.59%)


...

Cámara activada. Dibuja un rectángulo alrededor del objeto con el mouse y presiona 'c' para capturar.
Forma predicha: rhombus


¿Es correcto? (s/n, o 'q' para salir):  n


Selecciona la forma correcta (0-23): {'circle': 0, 'cone': 1, 'cube': 2, 'cuboid': 3, 'cylinder': 4, 'decagon': 5, 'dodecahedron': 6, 'ellipse': 7, 'heptagon': 8, 'hexagon': 9, 'nonagon': 10, 'octagon': 11, 'octahedron': 12, 'parallelogram': 13, 'pentagon': 14, 'prism': 15, 'pyramid': 16, 'rectangle': 17, 'rhombus': 18, 'sphere': 19, 'square': 20, 'tetrahedron': 21, 'trapezoid': 22, 'triangle': 23}


Ingresa el número de la forma correcta:  0


Cámara activada. Dibuja un rectángulo alrededor del objeto con el mouse y presiona 'c' para capturar.


Cargadas 19367 imágenes de 16 clases iniciales
Clases iniciales: {'circle': 0, 'cone': 1, 'cube': 2, 'cuboid': 3, 'cylinder': 4, 'ellipse': 5, 'hexagon': 6, 'octagon': 7, 'pentagon': 8, 'prism': 9, 'pyramid': 10, 'rectangle': 11, 'rhombus': 12, 'sphere': 13, 'square': 14, 'triangle': 15}
Cargadas 16 imágenes de 16 clases tras filtrado
Clases seleccionadas: ['circle', 'cone', 'cube', 'cuboid', 'cylinder', 'ellipse', 'hexagon', 'octagon', 'pentagon', 'prism', 'pyramid', 'rectangle', 'rhombus', 'sphere', 'square', 'triangle']
Entrenamiento: 12 imágenes, Validación: 4 imágenes
Entrenando nuevo modelo con todas las imágenes disponibles...
Epoch 1/30


D:\Descargas 2.0\SEMESTREVII\InteraccionHumanoComputador\PrimeraTarea\AnalisiOfSolidAndPlane\venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
D:\Descargas 2.0\SEMESTREVII\InteraccionHumanoComputador\PrimeraTarea\AnalisiOfSolidAndPlane\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 2.8052 - val_accuracy: 1.0000 - val_loss: 0.0522
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step - accuracy: 1.0000 - loss: 0.0741 - val_accuracy: 1.0000 - val_loss: 7.6294e-06
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step - accuracy: 1.0000 - loss: 2.3479e-04 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step - accuracy: 1.0000 - loss: 7.0532e-07 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step - accuracy: 1.0000 - loss: 0.0000e

Nuevo modelo guardado en: D:\Descargas 2.0\SEMESTREVII\InteraccionHumanoComputador\PrimeraTarea\AnalisiOfSolidAndPlane\data\models\modelo_formas.h5
Clases utilizadas: ['circle', 'cone', 'cube', 'cuboid', 'cylinder', 'ellipse', 'hexagon', 'octagon', 'pentagon', 'prism', 'pyramid', 'rectangle', 'rhombus', 'sphere', 'square', 'triangle']


In [2]:
import sys
import os
sys.path.append(os.path.abspath('..'))

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from src.utils.data_loader import load_data
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# Define la ruta de datos y del modelo
data_dir = os.path.join(os.path.dirname(os.path.abspath('')), 'data', 'raw')
model_path = os.path.join(os.path.dirname(os.path.abspath('')), 'data', 'models', 'modelo_formas.h5')

# Define las clases utilizadas
selected_classes = ['circle', 'cone', 'cube', 'cuboid', 'cylinder', 'ellipse', 'hexagon', 'octagon', 'pentagon', 'prism', 'pyramid', 'rectangle', 'rhombus', 'sphere', 'square', 'triangle']
num_classes = len(selected_classes)

# Carga el modelo
model = tf.keras.models.load_model(model_path)
print(f"Modelo cargado desde: {model_path}")

# Carga todos los datos disponibles (500 imágenes por clase)
images, labels, forma_to_label = load_data(data_dir, max_images_per_class=500)  # Usa todas las 500 por clase
print(f"Cargadas {len(images)} imágenes de {len(forma_to_label)} clases iniciales")
print("Clases iniciales:", forma_to_label)

# Filtra imágenes y etiquetas para las clases seleccionadas, asegurando todas las imágenes
images_selected = []
labels_selected = []
for i, forma in enumerate(forma_to_label):
    if forma in selected_classes:
        class_indices = [j for j, label in enumerate(labels) if label == i]
        images_selected.extend([images[j] for j in class_indices])
        labels_selected.extend([i] * len(class_indices))

# Crea un mapeo de índices originales a nuevos índices basados en selected_classes
selected_indices = [i for i, forma in enumerate(forma_to_label) if forma in selected_classes]
label_mapping = {i: idx for idx, i in enumerate(selected_indices)}
labels_mapped = [label_mapping.get(label, -1) for label in labels_selected]
labels_mapped = [label for label in labels_mapped if label != -1]
images_selected = [img for i, img in enumerate(images_selected) if i < len(labels_mapped)]
labels_mapped = labels_mapped[:len(images_selected)]  # Asegura misma longitud

print(f"Cargadas {len(images_selected)} imágenes de {len(selected_classes)} clases tras filtrado")
print("Clases seleccionadas:", selected_classes)

# Normaliza las imágenes
images_selected = np.array(images_selected) / 255.0
labels_one_hot = tf.keras.utils.to_categorical(labels_mapped, num_classes)

# Divide los datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(images_selected, labels_one_hot, test_size=0.2, random_state=42)
print(f"Entrenamiento: {len(X_train)} imágenes, Prueba: {len(X_test)} imágenes")

# Evalúa el modelo en el conjunto de prueba
predictions = model.predict(np.array(X_test))
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)

# Métricas con manejo de todas las clases
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='weighted', zero_division=0, labels=range(num_classes))
recall = recall_score(true_classes, predicted_classes, average='weighted', zero_division=0, labels=range(num_classes))
conf_matrix = confusion_matrix(true_classes, predicted_classes, labels=range(num_classes))

print(f"\nEvaluación del Modelo Actual:")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print("Matriz de Confusión:")
print(conf_matrix)

# Comparación con el modelo anterior
previous_accuracy = 0.53  # Valor del modelo anterior con 1000 imágenes de 20 clases
print(f"\nComparación con el Modelo Anterior:")
print(f"Accuracy Anterior: {previous_accuracy:.2f}")
print(f"Accuracy Actual: {accuracy:.2f}")
if accuracy > previous_accuracy:
    print("El modelo actual mejoró en comparación con el anterior.")
elif accuracy < previous_accuracy:
    print("El modelo actual empeoró en comparación con el anterior.")
else:
    print("El modelo actual tiene el mismo rendimiento que el anterior.")

# Mejora si la precisión es baja
if accuracy < 0.8 and len(images_selected) > num_classes:
    print("\nPrecisión baja. Iniciando reentrenamiento...")
    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    train_generator = datagen.flow(np.array(X_train), y_train, batch_size=32)
    validation_generator = datagen.flow(np.array(X_val), y_val, batch_size=32)

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_generator, epochs=20, validation_data=validation_generator, verbose=1)

    # Guarda el modelo mejorado
    model.save(model_path)
    print(f"Modelo mejorado guardado en: {model_path}")
else:
    print("\nPrecisión aceptable o datos insuficientes. No se requiere reentrenamiento.")

Modelo cargado desde: D:\Descargas 2.0\SEMESTREVII\InteraccionHumanoComputador\PrimeraTarea\AnalisiOfSolidAndPlane\data\models\modelo_formas.h5
Cargadas 4695 imágenes de 16 clases iniciales
Clases iniciales: {'circle': 0, 'cone': 1, 'cube': 2, 'cuboid': 3, 'cylinder': 4, 'ellipse': 5, 'hexagon': 6, 'octagon': 7, 'pentagon': 8, 'prism': 9, 'pyramid': 10, 'rectangle': 11, 'rhombus': 12, 'sphere': 13, 'square': 14, 'triangle': 15}
Cargadas 4695 imágenes de 16 clases tras filtrado
Clases seleccionadas: ['circle', 'cone', 'cube', 'cuboid', 'cylinder', 'ellipse', 'hexagon', 'octagon', 'pentagon', 'prism', 'pyramid', 'rectangle', 'rhombus', 'sphere', 'square', 'triangle']
Entrenamiento: 3756 imágenes, Prueba: 939 imágenes
30/30 ━━━━━━━━━━━━━━━━━━━━ 6s 47ms/step

Evaluación del Modelo Actual:
Accuracy: 0.87
Precision: 0.88
Recall: 0.87
Matriz de Confusión:
[[115   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0  24   0   0   0   0   0   0   0   0   7   0   2   7   0   0]
 [  0